In [53]:
# Instalamos librerias externas
%pip install redis-server redis numpy openai\[datalib\] -q

Note: you may need to restart the kernel to use updated packages.


In [54]:
# Importamos librerias
import redis
import openai
import getpass
import time
import numpy as np

In [31]:
# Varianles globales
openai.api_key = getpass.getpass()
model = 'text-embedding-ada-002'

In [55]:
# creamos un cliente de redis
class RedisClient:
    def __init__(self, host='localhost', port=6379):
        self.client = redis.Redis(host=host, port=port)

    def set(self, key, value):
        try:
            return self.client.set(key, value)
        except ConnectionError:
            print("Connection Error")

    def get(self, key):
        try:
            return self.client.get(key)
        except ConnectionError:
            print("Connection Error")

    def delete(self, key):
        try:
            return self.client.delete(key)
        except ConnectionError:
            print("Connection Error")

    def keys(self, pattern='*'):
        try:
            return self.client.keys(pattern)
        except ConnectionError:
            print("Connection Error")

    
    def exists(self, key):
        try:
            return self.client.exists(key)
        except ConnectionError:
            print("Connection Error")
            

    def flushall(self):
        try:
            return self.client.flushall()
        except ConnectionError:
            print("Connection Error")

    def flushdb(self):
        try:
            return self.client.flushdb()
        except ConnectionError:
            print("Connection Error")

In [61]:
# retrieve the embedding from the redis cache
# if it doesn't exist, create it and store it in the cache
# then return it

def get_embedding(text):
    # create a redis client
    redis_client = RedisClient()

    # get the embedding from the cache
    embeddings = redis_client.get(text)

    # if the embedding doesn't exist in the cache
    if embeddings is None:
        # create the embedding
        time.sleep(1)
        print('Creating embedding for: ' + text)
        
        response = openai.Embedding.create(
            input = text,
            model = model
        )

        print(response['data'][0]['embedding'])
        embeddings = response['data'][0]['embedding']
        
        # store the embedding in the cache
        redis_client.set(text, embeddings)
    # return the embedding
    return embeddings

In [62]:
# Ejemplo de funcionamiento
get_embedding("This is a test")

# openai.Engine.list() 

Creating embedding for: This is a test


MissingDependencyError: 

OpenAI error:

    missing `numpy`

This feature requires additional dependencies:

    $ pip install openai[datalib]

